### Implementing BPE algorithm

1. Undergoes the normalization and pre-tokenization stage.
2. The normalization step involves some general cleanup, such as removing needless whitespace, lowercasing, and/or removing accents.
3. Pre-tokenization split the texts into small entities and the way they do this can be different for different algorithms.
4. Get the list of all the unique words in the text corpus.
5. Build the vocabulary using the characters used in each words.

    ```
    If the corpus : ["hug", "pug", "pun", "bun", "hugs"]

    vocabulary : ["b", "g", "h", "n", "p", "s", "u"]
    ```

6. Now we have a hyperparameter to set, the desired vocab size. To reach this desired size, we keep merging common tokens.
7. Repeat

In [12]:
corpus = [
    "Hi my name is Jino",
    "I am sike years old",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
    "Actually my name is Rohit"
]

In [ ]:
from collections import defaultdict

class BPE:
    def __init__(self, corpus, vocab_size=50, debug=False):
        self.corpus = corpus
        self.vocab_size = vocab_size
        self.debug = debug
        self.word_freqs = defaultdict(int)
        self.splits = {}
        self.merges = defaultdict(int)
        self.vocab = ["<|endoftext|>"]
        self.alphabet = []

    def _normalization_pre_tokenization(self):
        # normalize to lowercase and split on empty spaces
        self.corpus = [word.lower() for sent in self.corpus for word in sent.split(' ')]
        if self.debug:
            print("Normalized Corpus:", self.corpus)
            print('\n')

    def _get_frequency(self):
        # Compute word frequencies and initial splits
        for word in self.corpus:
            self.word_freqs[word] += 1
        self.splits = {word: list(word) for word in self.word_freqs.keys()}

    def _base_vocab(self):
        # Generate the base alphabet vocabulary
        self.alphabet = sorted({letter for word in self.word_freqs.keys() for letter in word})
        self.vocab += self.alphabet.copy()

    def compute_pair_freqs(self):
        # Compute frequencies of adjacent pairs in current splits
        pair_freqs = defaultdict(int)
        for word, freq in self.word_freqs.items():
            split = self.splits[word]
            for i in range(len(split) - 1):
                pair = (split[i], split[i + 1])
                pair_freqs[pair] += freq
        return pair_freqs

    def merge_pair(self, a, b):
        # Merge the most frequent pair in all words
        for word in self.splits.keys():
            split = self.splits[word]
            i = 0
            while i < len(split) - 1:
                if split[i] == a and split[i + 1] == b:
                    split = split[:i] + [a + b] + split[i + 2:]
                else:
                    i += 1
            self.splits[word] = split

    def train(self):
        self._normalization_pre_tokenization()
        self._get_frequency()
        self._base_vocab()

        while len(self.vocab) < self.vocab_size:
            pair_freqs = self.compute_pair_freqs()
            if not pair_freqs:
                break

            best_pair = max(pair_freqs, key=pair_freqs.get)
            new_token = ''.join(best_pair)
            if self.debug:
                print(f"Best Pair to Merge: {best_pair} with Frequency: {pair_freqs[best_pair]}")
            
            self.merge_pair(*best_pair)
            self.merges[best_pair] = new_token
            self.vocab.append(new_token)

            if self.debug:
                print("Updated Vocabulary:", self.vocab)
                print('#################################')
                print('\n')

        if self.debug:
            print("Final Vocabulary:", self.vocab)

In [53]:
bpe = BPE(corpus, vocab_size = 50, debug = True)
bpe.train()

Normalized Corpus: ['hi', 'my', 'name', 'is', 'jino', 'i', 'am', 'sike', 'years', 'old', 'this', 'section', 'shows', 'several', 'tokenizer', 'algorithms.', 'hopefully,', 'you', 'will', 'be', 'able', 'to', 'understand', 'how', 'they', 'are', 'trained', 'and', 'generate', 'tokens.', 'actually', 'my', 'name', 'is', 'rohit']


Best Pair to Merge: ('e', 'r') with Frequency: 4
Updated Vocabulary: ['<|endoftext|>', ',', '.', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'er']
#################################


Best Pair to Merge: ('h', 'i') with Frequency: 3
Updated Vocabulary: ['<|endoftext|>', ',', '.', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'er', 'hi']
#################################


Best Pair to Merge: ('a', 'm') with Frequency: 3
Updated Vocabulary: ['<|endoftext|>', ',', '.', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k',